In [1]:
#Import Packages
import pandas as pd
import numpy as np
import os
import itertools
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import datetime
import matplotlib
from herbie import Herbie
import pickle
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
import pysolar.solar as solar
from geographiclib.geodesic import Geodesic
import xarray as xr
import pytz
import simplekml
import cartopy
import sys
sys.path.append('..')
import funcs.ac_funcs as ac
import funcs.plotting as plotting


pd.options.mode.chained_assignment = None


In [19]:
base_project_dir = '/Users/agmeyer4/LAIR_1/'

#define datetime range and interval parameters
dt1_str = '2022-11-15 00:00:00'#start datetime
dt2_str = '2022-11-15 23:00:00' #end datetime
timezone = 'UTC' #this should nearly always be UTC, but can identify if your datetime strings above are in a different timezone
dt1 = ac.dtstr_to_dttz(dt1_str,timezone)
dt2 = ac.dtstr_to_dttz(dt2_str,timezone)

oof_data_folder_cindy = os.path.join(base_project_dir,'Data/EM27_oof/Test_ua_cindy/')
my_oof_manager_cindy = ac.oof_manager(oof_data_folder_cindy,timezone)
oof_df_cindy = my_oof_manager_cindy.load_oof_df_inrange(dt1,dt2)

oof_data_folder_agm = os.path.join(base_project_dir,'Data/EM27_oof/Test_ua_agm/')
my_oof_manager_agm = ac.oof_manager(oof_data_folder_agm,timezone)
oof_df_agm = my_oof_manager_agm.load_oof_df_inrange(dt1,dt2)

In [20]:
# xlims= [datetime.datetime.combine(em27_resampled.index[0].date(),datetime.time(hour=6)),
#         datetime.datetime.combine(em27_resampled.index[-1].date(),datetime.time(hour=22))]
spec = 'xco2(ppm)'
fig = go.Figure()
fig.add_trace(go.Scatter(
        x = oof_df_cindy.index,
        y = oof_df_cindy[spec],
        name='Cindy',
        # error_y = dict(
        #     type='data', # value of error bar given in data coordinates
        #     array=oof_df_cindy[f'{spec}_error'],
        #     visible=True,
        #     color = 'grey',
        #     width = 0.8),
        mode='markers',
        marker={'color':'black'},
    ))
fig.add_trace(go.Scatter(
        x = oof_df_agm.index,
        y = oof_df_agm[spec],
        name='agm',
        # error_y = dict(
        #     type='data', # value of error bar given in data coordinates
        #     array=oof_df_agm[f'{spec}_error'],
        #     visible=True,
        #     color = 'pink',
        #     width = 0.1),
        mode='markers',
        marker={'color':'red','size':2},
    ))
fig.update_yaxes(title_text = spec)



fig.update_layout(
    height=400,
)
fig.show()

In [15]:
oof_df_agm.compare(oof_df_cindy)

dt
